In [1]:
import pandas as pd
import geopandas as gpd

df = pd.read_csv('traffic_detection.csv')
roads = pd.read_csv('Calgary_Roads.csv')

display(df.head())
display(roads.head())

,Camera Number,car,truck,bus,quadrant,camera_location,Camera_Number,URL,longitude,latitude
0,1,6.0,2.0,0.0,SW/SE,Glenmore Trail / Macleod Trail S,1,http://trafficcam.calgary.ca/loc0.jpg,-114.071585,50.993989
1,2,8.0,0.0,0.0,SE,Peigan Trail / Barlow Trail SE,2,http://trafficcam.calgary.ca/loc1.jpg,-114.002689,51.014972
2,3,3.0,0.0,0.0,SE,50 Avenue / Barlow Trail SE,3,http://trafficcam.calgary.ca/loc2.jpg,-114.001845,51.008623
3,4,9.0,1.0,1.0,NE,Memorial Drive / 52 Street E,4,http://trafficcam.calgary.ca/loc3.jpg,-113.955818,51.053253
4,5,3.0,1.0,0.0,NW,Kensington Road / Crowchild Trail NW,5,http://trafficcam.calgary.ca/loc4.jpg,-114.118088,51.052487


,the_geom,MODIFIED_ATTRIBUTE_DT,OBSOLETE_CODE,MODIFIED_GEOMETRY_DT,SEGMENT_ID,LEFT_FROM_ADDR,LEFT_TO_ADDR,RIGHT_FROM_ADDR,RIGHT_TO_ADDR,NUMERIC_PREFACE,...,SUBCLASS_SURFACE_TYPE,COMMENTS,FULL_NAME,NAME_VALIDATED,ONE_WAY,BUILT_STATUS,OWNERSHIP,AQUISITION_TECHNIQUE,ASSET_OWNERSHIP,ATTRIBUTE_LINKED
0,"LINESTRING (-113.965023344204 50.932622852311,...",NaN,No,NaN,244686,0,0,13048,12850,NaN,...,NaN,NaN,48 ST SE,Y,FT,Built,Corporate,Unknown,Unknown,No
1,"LINESTRING (-114.120037955152 51.143816565749,...",NaN,No,NaN,141793,0,0,2880,2898,NaN,...,NaN,NaN,COUNTRY HILLS BV NW,NaN,FT,Built,Corporate,Unknown,Unknown,No
2,"LINESTRING (-113.981662499694 51.052269470329,...",NaN,No,NaN,311099,0,0,0,0,NaN,...,NaN,NaN,36 ST SE,NaN,FT,Built,Corporate,Digital Plan,Roads,Yes
3,"LINESTRING (-114.187662271487 51.035454294065,...",NaN,No,NaN,222479,0,0,2001,2689,NaN,...,NaN,NaN,69 ST SW,Y,FT,Built,Corporate,Unknown,Unknown,No
4,"LINESTRING (-113.974007383887 50.957348170487,...",NaN,No,NaN,236712,0,0,10748,10740,NaN,...,NaN,NaN,42 ST SE,Y,FT,Built,Corporate,Unknown,Unknown,No


In [2]:
import shapely as shape
import geopandas as gpd
import numpy as np

#Create Geospatial dataframe
roads['geometry'] = roads['the_geom'].apply(shape.wkt.loads)

roads = gpd.GeoDataFrame(roads)
gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df['longitude'], df['latitude'], crs="EPSG:4326"), data=df)

In [11]:
import plotly.graph_objects as go

lats = []
lons = []
names = []

for feature, name in zip(roads['geometry'], roads['FULL_NAME']):
    if isinstance(feature, shape.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shape.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

In [ ]:
import plotyly.express as px
fig = px.line_mapbox(lat=lats, lon=lons, hover_name=names,
                     mapbox_style="open-street-map", zoom=1)

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = gdf['longitude'],
    lat = gdf['latitude'],
    marker = {'size': 5,'color': 'black'}))
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    height = 800, width =500,
    mapbox = {
        'center': {'lon': -114.065353, 'lat': 51.066886},
        'style': "open-street-map",
        'center': {'lon': -114.065353, 'lat': 51.066886},
        'zoom': 9})

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = gdf['longitude'],
    lat = gdf['latitude'],
    marker = {'size': 10}))


for feature, name in zip(roads['geometry'], roads['FULL_NAME']):
    if isinstance(feature, shape.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shape.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = list(y)
        lons = list(x)        
        fig.add_trace(go.Scattermapbox(
            mode = "lines",
            lon = lons,
            lat = lats,
            marker = {'color': 'blue'}))
        
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    height = 800, width =500,
    mapbox = {
        'center': {'lon': -114.065353, 'lat': 51.066886},
        'style': "open-street-map",
        'center': {'lon': -114.065353, 'lat': 51.066886},
        'zoom': 9})

fig.show()

In [7]:
import pandas as pd
chart_data = pd.read_csv('traffic_detection-2.csv')
chart_data.rename(columns = {'camera_location': 'Intersection'}, inplace = True)
chart_data['traffic_sum'] = chart_data[['car','truck','bus']].sum(axis = 1)
chart_data['Traffic'] = pd.cut(chart_data['traffic_sum'], 
                            bins = chart_data['traffic_sum'].quantile([0,.2, .8,1]),
                            labels = ['Low', 'Moderate', 'High'])

In [13]:
list(chart_data['Traffic'].dropna().unique())
#list(chart_data['Traffic'].value_counts())

['Moderate', 'Low', 'High']

In [15]:
pd.date_range('2024-04-01', periods=24*12, freq='5min')

DatetimeIndex(['2024-04-01 00:00:00', '2024-04-01 00:05:00',
               '2024-04-01 00:10:00', '2024-04-01 00:15:00',
               '2024-04-01 00:20:00', '2024-04-01 00:25:00',
               '2024-04-01 00:30:00', '2024-04-01 00:35:00',
               '2024-04-01 00:40:00', '2024-04-01 00:45:00',
               ...
               '2024-04-01 23:10:00', '2024-04-01 23:15:00',
               '2024-04-01 23:20:00', '2024-04-01 23:25:00',
               '2024-04-01 23:30:00', '2024-04-01 23:35:00',
               '2024-04-01 23:40:00', '2024-04-01 23:45:00',
               '2024-04-01 23:50:00', '2024-04-01 23:55:00'],
              dtype='datetime64[ns]', length=288, freq='5min')

In [16]:
np.arange(12)

NameError: name 'np' is not defined